In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import signal 
import pickle


import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
dataframe_hrv = pd.read_csv("C:/Users/SplendidUnivers/Desktop/dataset/dataframe_hrv.csv")
dataframe_hrv = dataframe_hrv.reset_index(drop=True)

In [4]:
display(dataframe_hrv.head(5))

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
0,-0.001974,-0.004737,77.815789,10.801842,12.529684,2.417132,10.889447,0.614632,NaN,12.529684,0.0,12.529684,0.973684,0.617297,3.558630e-02,0.015203,0.055556,0.001238,0.0,0.000696,0.000407,0.000135,3.00200
1,0.002935,-0.004457,101.978261,10.750609,30.503500,2.417109,11.251065,0.647826,NaN,30.503500,0.0,30.503500,0.978261,0.647889,1.354660e-02,0.013858,0.045455,0.000144,0.0,0.000009,0.000060,0.000075,0.79371
2,0.006745,-0.003426,104.957447,10.557234,52.523021,2.226872,11.379638,0.646383,NaN,52.523021,0.0,52.523021,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
3,-0.004043,-0.002532,87.702128,10.640128,74.402170,2.173021,11.470830,0.645000,NaN,74.402170,0.0,74.402170,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN
4,0.012745,-0.004426,88.829787,10.699319,96.219617,2.017106,11.135255,0.645000,NaN,96.219617,0.0,96.219617,0.978723,0.645000,2.240000e-08,0.000000,0.000000,NaN,0.0,NaN,NaN,NaN,NaN


In [5]:
display(dataframe_hrv.describe())
print(dataframe_hrv.columns)

,ECG,EMG,HR,RESP,Seconds,footGSR,handGSR,interval in seconds,marker,newtime,stress,time,NNRR,AVNN,SDNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
count,4129.000000,4023.000000,4129.000000,4129.000000,4129.000000,4129.000000,4056.000000,4129.000000,3722.000000,4129.000000,4129.000000,4129.000000,4129.000000,4129.000000,4.007000e+03,4129.000000,4129.000000,3830.000000,3957.000000,4002.000000,4002.000000,4002.000000,58.000000
mean,0.170927,0.604475,83.136251,29.846928,2278.789710,6.487689,11.523244,0.788440,12.057030,35171.533570,0.516078,2278.789710,0.975377,0.788020,5.781201e-02,0.027223,0.034246,inf,inf,0.000728,0.000003,0.000001,3.555695
std,0.137222,0.807892,17.633319,14.024560,1313.496347,4.307487,6.739658,0.109768,2.612543,19946.879538,0.431116,1313.496347,0.002469,0.109363,3.401004e-02,0.016964,0.025153,NaN,NaN,0.003244,0.000031,0.000010,2.080121
min,-0.699585,-0.697800,11.800000,-12.606244,12.529684,0.971111,-28.381533,0.527944,0.000000,12.529684,0.000000,12.529684,0.970588,0.528170,7.840000e-09,0.000000,0.000000,0.000008,0.000000,0.000000,0.000000,0.000000,0.418526
25%,0.084500,0.108651,72.861111,28.565064,1114.941256,2.800841,6.713688,0.718930,10.350161,18158.311120,0.000000,1114.941256,0.973684,0.720548,3.187020e-02,0.013743,0.025641,0.014617,0.009989,0.000000,0.000000,0.000000,2.220580
50%,0.144171,0.302947,79.926829,32.916500,2295.794860,5.770098,9.651417,0.789732,11.070972,34158.425430,0.631579,2295.794860,0.975000,0.790125,4.637430e-02,0.022889,0.027027,0.034332,0.028017,0.000000,0.000000,0.000000,3.169260
75%,0.207457,0.822690,89.400000,38.627444,3425.276711,8.837477,16.647151,0.869053,12.879199,52300.114070,0.974359,3425.276711,0.977273,0.868135,8.941850e-02,0.043000,0.030303,0.105621,0.105178,0.000000,0.000000,0.000000,4.865465
max,0.681714,9.491700,372.000000,52.089590,5005.373902,22.582463,31.219800,1.040088,35.624000,70173.851200,1.000000,5005.373902,0.981481,1.038360,1.645780e-01,0.080981,0.257143,inf,inf,0.035841,0.000616,0.000261,9.117240


Index(['ECG', 'EMG', 'HR', 'RESP', 'Seconds', 'footGSR', 'handGSR',
       'interval in seconds', 'marker', 'newtime', 'stress', 'time', 'NNRR',
       'AVNN', 'SDNN', 'RMSSD', 'pNN50', 'TP', 'ULF', 'VLF', 'LF', 'HF',
       'LF_HF'],
      dtype='object')


In [6]:
def fix_stress_labels(df='',label_column='stress'):
    df['stress'] = np.where(df['stress']>=0.5, 1, 0)
    display(df["stress"].unique())
    return df
dataframe_hrv = fix_stress_labels(df=dataframe_hrv)

array([0, 1], dtype=int64)

In [54]:
def missing_values(df):
    df = df.reset_index()
    df = df.replace([np.inf, -np.inf], np.nan)
    df[~np.isfinite(df)] = np.nan
#     df.plot( y=["HR"])
    df['HR'].fillna((df['HR'].mean()), inplace=True)
    df['HR'] = signal.medfilt(df['HR'],13) 
#     df.plot( y=["HR"])

    df=df.fillna(df.mean(),inplace=True)
    return df

dataframe_hrv = missing_values(dataframe_hrv)


In [20]:
import random
from scipy.signal import resample

selected_x_columns = ['HR','interval in seconds','AVNN', 'RMSSD', 'pNN50', 'TP', 'ULF', 'VLF', 'LF', 'HF','LF_HF']

X = dataframe_hrv[selected_x_columns]
y = dataframe_hrv['stress']

display(X.columns)
display(X.describe())
display(X.shape)

#data augmentation
def stretch(x):
    l = int(11 * (1 + (random.random()-0.5)/3))
    y = resample(x, l)
    if l < 11:
        y_ = np.zeros(shape=(11, ))
        y_[:l] = y
    else:
        y_ = y[:11]
    return y_

def amplify(x):
    alpha = (random.random()-0.5)
    factor = -alpha*x + (1+alpha)
    return x*factor

def augment(x):
    result = np.zeros(shape= (4, 11))
    for i in range(3):
        if random.random() < 0.33:
            new_y = stretch(x)
        elif random.random() < 0.66:
            new_y = amplify(x)
        else:
            new_y = stretch(x)
            new_y = amplify(new_y)
        result[i, :] = new_y
    return result

#data storage into results and classes
result = np.apply_along_axis(augment, axis=1, arr=X).reshape(-1, 11)
classe = np.ones(shape=(result.shape[0],), dtype=int)*3
X = np.vstack([X, result])
y = np.hstack([y, classe])

train_features, test_features, train_labels, test_labels = train_test_split(X, y,test_size=0.25)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

train_features = np.expand_dims(train_features, 2)
test_features = np.expand_dims(test_features, 2)


#apply onehotencoder to reshape the datasets
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
train_labels = ohe.fit_transform(train_labels.reshape(-1,1))
test_labels = ohe.transform(test_labels.reshape(-1,1))

print("X_train", train_features.shape)
print("y_train", train_labels.shape)
print("X_test", test_features.shape)
print("y_test", test_labels.shape)

Index(['HR', 'interval in seconds', 'AVNN', 'RMSSD', 'pNN50', 'TP', 'ULF',
       'VLF', 'LF', 'HF', 'LF_HF'],
      dtype='object')

,HR,interval in seconds,AVNN,RMSSD,pNN50,TP,ULF,VLF,LF,HF,LF_HF
count,4129.000000,4129.000000,4129.000000,4129.000000,4129.000000,3830.000000,3957.000000,4002.000000,4002.000000,4002.000000,58.000000
mean,83.136251,0.788440,0.788020,0.027223,0.034246,inf,inf,0.000728,0.000003,0.000001,3.555695
std,17.633319,0.109768,0.109363,0.016964,0.025153,NaN,NaN,0.003244,0.000031,0.000010,2.080121
min,11.800000,0.527944,0.528170,0.000000,0.000000,0.000008,0.000000,0.000000,0.000000,0.000000,0.418526
25%,72.861111,0.718930,0.720548,0.013743,0.025641,0.014617,0.009989,0.000000,0.000000,0.000000,2.220580
50%,79.926829,0.789732,0.790125,0.022889,0.027027,0.034332,0.028017,0.000000,0.000000,0.000000,3.169260
75%,89.400000,0.869053,0.868135,0.043000,0.030303,0.105621,0.105178,0.000000,0.000000,0.000000,4.865465
max,372.000000,1.040088,1.038360,0.080981,0.257143,inf,inf,0.035841,0.000616,0.000261,9.117240


(4129, 11)

Training Features Shape: (15483, 11)
Training Labels Shape: (15483,)
Testing Features Shape: (5162, 11)
Testing Labels Shape: (5162,)
X_train (15483, 11, 1)
y_train (15483, 3)
X_test (5162, 11, 1)
y_test (5162, 3)


C:\Users\SplendidUnivers\Miniconda3\envs\test\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [24]:
from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Softmax, Add, Flatten, Activation# , Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import math

#the Model
n_obs, feature, depth = train_features.shape
batch_size = 300


inp = Input(shape=(feature, depth))
C = Conv1D(filters=32, kernel_size=3, strides=1)(inp)

C11 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(C)
A11 = Activation("relu")(C11)
C12 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(A11)
S11 = Add()([C12, C])
A12 = Activation("relu")(S11)
M11 = MaxPooling1D(pool_size=3, strides=1)(A12)


C21 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(M11)
A21 = Activation("relu")(C21)
C22 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(A21)
S21 = Add()([C22, M11])
A22 = Activation("relu")(S11)
M21 = MaxPooling1D(pool_size=3, strides=1)(A22)


C31 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(M21)
A31 = Activation("relu")(C31)
C32 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(A31)
S31 = Add()([C32, M21])
A32 = Activation("relu")(S31)
M31 = MaxPooling1D(pool_size=3, strides=1)(A32)


C41 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(M31)
A41 = Activation("relu")(C41)
C42 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(A41)
S41 = Add()([C42, M31])
A42 = Activation("relu")(S41)
M41 = MaxPooling1D(pool_size=3, strides=1)(A42)


C51 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(M41)
A51 = Activation("relu")(C51)
C52 = Conv1D(filters=32, kernel_size=3, strides=1, padding='same')(A51)
S51 = Add()([C52, M41])
A52 = Activation("relu")(S51)
M51 = MaxPooling1D(pool_size=3, strides=1)(A52)

F1 = Flatten()(M51)

D1 = Dense(32)(F1)
A6 = Activation("relu")(D1)
D2 = Dense(32)(A6)
D3 = Dense(3)(D2)
A7 = Softmax()(D3)

model = Model(inputs=inp, outputs=A7)

model.summary()

#compiling and fitting the model
def exp_decay(epoch):
    initial_lrate = 0.001
    k = 0.75
    t = n_obs//(10000 * batch_size)  # every epoch we do n_obs/batch_size iteration
    lrate = initial_lrate * math.exp(-k*t)
    return lrate

lrate = LearningRateScheduler(exp_decay)
adam = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(train_features, train_labels, 
                    epochs=75, 
                    batch_size=batch_size, 
                    verbose=2, 
                    validation_data=(test_features, test_labels), 
                    callbacks=[lrate])

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 11, 1)        0                                            
__________________________________________________________________________________________________
conv1d_54 (Conv1D)              (None, 9, 32)        128         input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_55 (Conv1D)              (None, 9, 32)        3104        conv1d_54[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 9, 32)        0           conv1d_55[0][0]                  
____________________________________________________________________________________________

Epoch 12/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 13/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 14/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 15/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 16/75
 - 2s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 17/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 18/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 19/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 20/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 21/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 22/75
 - 1s - loss: nan - accuracy: 0.0926 - val_loss: nan - val_accuracy: 0.1019
Epoch 23/75
 - 1s - loss: nan - 